In [1]:
# En esta celda se puede probar la funcion de analisis de una imagen

from lib.image import image_data
from lib.analysis import dyssynchrony_analysis
import cv2

# Path completo donde esta la imagen (incluyendo nombre y extencion)
path = 'photos_examples/c1d000.tif'

# Ancho de la feta a analisar (es un parametro opcional)
ancho_corte = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 20

# Calibracion del tiempo de cada pixel
calibration = 3.1

image = cv2.imread(path)
# Select ROI
fromCenter = False
showCrosshair = False
seleted_parameters = cv2.selectROI(image, fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.destroyAllWindows()

image = image_data.get_image_data(image)
image = image_data.crop_vertical(image, x_start, x_end)
image = image_data.crop_horizontal(image, y_start, y_end)

results = dyssynchrony_analysis.analyze_image(image, min_dist_between_maxs, calibration, slice_width=ancho_corte)
results

/home/agusgs/Projects/CardiAP/DesktopApp/lib/analysis/dyssynchrony_analysis.py:45: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  (w, residuals, rank, sing_vals) = lstsq(np.asarray([[1, t] for t in times]), np.asarray(bList).T)
/home/agusgs/Projects/CardiAP/DesktopApp/venv/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


{'image': {'intensities': array([26, 28, 30, 32, 34, 37, 38, 40, 42, 44, 45, 46, 47, 48, 48, 48, 47,
         47, 46, 46, 45, 44, 44, 43, 42, 42, 41, 40, 39, 38, 37, 37, 36, 35,
         35, 34, 33, 33, 32, 31, 31, 30, 30, 29, 29, 28, 28, 27, 27, 26, 25,
         25, 25, 24, 24, 23, 23, 22, 22, 22, 21, 21, 21, 20, 20, 19, 19, 19,
         19, 18, 18, 18, 19, 20, 21, 22, 24, 25, 27, 29, 31, 33, 35, 37, 39,
         41, 42, 44, 45, 46, 46, 47, 47, 47, 46, 46, 45, 45, 44, 43, 43, 42,
         42, 41, 40, 39, 39, 38, 37, 37, 36, 35, 35, 34, 33, 33, 32, 32, 31,
         30, 30, 29, 29, 28, 28, 27, 27, 26, 26, 25, 25, 24, 24, 23, 23, 22,
         22, 21, 21, 21, 20, 20, 20, 19, 19, 18, 18, 18, 17, 17, 18, 19, 20,
         21, 22, 24, 26, 28, 29, 32, 34, 36, 38, 40, 41, 43, 45, 46, 47, 47,
         47, 47, 47, 47, 46, 46, 45, 45, 44, 44, 43, 42, 42, 41, 40, 39, 39,
         38, 37, 37, 36, 35, 34, 34, 33, 32, 32, 31, 30, 30, 29, 29, 28, 28,
         27, 27, 26, 26, 25, 25, 25, 24, 24, 23, 23,

# PRUEBA ANALISIS 

In [ ]:
# En esta celda se puede probar la funcion de analisis todas las imagenes .tif en un directorio
import os
from lib.image import image_data
from lib.analysis import dyssynchonia_analysis
import cv2

# Ancho de la feta a analisar (es un parametro opcional)
ancho_corte = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 20


# Path donde estan las imagenes
path = "C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/seleccion/050320sel/050320c1"

images_paths = [ f'{path}/{file}' for file in os.listdir(path) if file.endswith(".tif") ]
images = [ cv2.imread(image_path) for image_path in images_paths ]

In [ ]:
fromCenter = False
showCrosshair = False

#Tomamos la primer imagen para seleccionar el recorte elegido el resto de las imagenes se van a recortar igual
seleted_parameters = cv2.selectROI(images[0], fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.destroyAllWindows()

In [ ]:
images = [ image_data.get_image_data(image) for image in images ]
images = [ image_data.crop_vertical(image, x_start, x_end) for image in images ]
images = [ image_data.crop_horizontal(image, y_start, y_end) for image in images ]
images

In [ ]:
[dyssynchonia_analysis.analyze_image(image, min_dist_between_maxs, ancho_corte) for image in images]

# Analisis de datos
Notas: 
- Image corresponde al análisis de toda la célula y slices corresponde a fetas de esa imagen
- intensidades es el valor que resulta de comprimir (sumando) la matriz con las coordenadas de x,y,z de los pixeles 
- max_peaks_pos es la index de en la lista de intensidades 
- max_peaks_intensities el valor de intensidad que se corresponde con el pico en la posicion analoga de max_peaks_pos

In [ ]:
#Imports
import json
import numpy as np
import pandas as pd

In [ ]:
#encodes a dictionary into a jason
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def jasonGenerator(path,results):
    """ Generates a .jason file in the path given folder from the a dictionary """
    with open(path + '/analysis_result.jason',"w") as miarch:
        miarch.write(json.dumps(reslust, cls=NumpyEncoder))

In [ ]:
def jasonReader(path):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open( path + '/analysis_result.json','r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [ ]:
dicto1 = jasonReader('/home/ana/CardiAp/DesktopApp')

In [ ]:
#dicto1['slices'][0]
dicto1['image']